In [1]:
from neo.io import AxonIO

fname = './ChatCreChR2/160120/16120020.abf'

r = AxonIO(filename=fname)
block = r.read()[0] # read the entire file > a Block
print(block)

In [23]:
import numpy as np
import matplotlib.pyplot as pl
pl.plot(np.histogram(block.segments[2].analogsignals))

ValueError: setting an array element with a sequence.

In [3]:
asig = block.segments[2].analogsignals
times = asig.times.rescale('s').magnitude
asig = asig.rescale('pA').magnitude
plot(times, asig)

AttributeError: 'list' object has no attribute 'times'

In [ ]:
import numpy as np
import quantities as pq
from matplotlib import pyplot

fig = pyplot.figure()

for seg in block.segments:
    #fig = pyplot.figure()
    #ax1 = fig.add_subplot(2, 1, 1)
    #ax2 = fig.add_subplot(2, 1, 2)
    #ax1.set_title(seg.file_origin)
    #mint = 0 * pq.s
    #maxt = np.inf * pq.s
    #for i, asig in enumerate(seg.analogsignals):
    #    times = asig.times.rescale('s').magnitude
    #    asig = asig.rescale('pA').magnitude
    #    ax1.plot(times, asig)

    #trains = [st.rescale('s').magnitude for st in seg.spiketrains]
    #colors = pyplot.cm.jet(np.linspace(0, 1, len(seg.spiketrains)))
    #ax2.eventplot(trains, colors=colors)

pyplot.show()

In [20]:
avg = np.mean([seg.analogsignals[5] for seg in block.segments[::2]], axis=1)
plt.plot(avg)

IndexError: list index out of range